In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'C0PWVFTOTWSJ0V4CCDVKZOOB2ZNKWS21O2JHHQLFSFRUAXQI' # your Foursquare ID
CLIENT_SECRET = 'LUWI2FWUHN2YRC5HCOYSF24R5JZNTOYTD3XBGSMDD1EC0P5I' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C0PWVFTOTWSJ0V4CCDVKZOOB2ZNKWS21O2JHHQLFSFRUAXQI
CLIENT_SECRET:LUWI2FWUHN2YRC5HCOYSF24R5JZNTOYTD3XBGSMDD1EC0P5I


### Converting DZ Bank Building address to its latitude and longitude coordinates.

In [3]:
address = '609 5th Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7577093 -73.9775923


### Define a query to search for Greek food that is within 500 metres from the DZ Bank Building

In [6]:
search_query = 'Greek'
radius = 500
print(search_query + ' .... OK!')

Greek .... OK!


In [7]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=C0PWVFTOTWSJ0V4CCDVKZOOB2ZNKWS21O2JHHQLFSFRUAXQI&client_secret=LUWI2FWUHN2YRC5HCOYSF24R5JZNTOYTD3XBGSMDD1EC0P5I&ll=40.7577093,-73.9775923&v=20180604&query=Greek&radius=500&limit=30'

#### Send the GET Request and examine the results

In [ ]:
results = requests.get(url).json()
results

In [9]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,5414bab4498e79ad23ecce9b,GRK Fresh Greek,"[{'id': '4bf58dd8d48988d10e941735', 'name': 'G...",v-1593680805,False,451 Lexington Ave,44th Street,40.753128,-73.974666,"[{'label': 'display', 'lat': 40.753128, 'lng':...",...,NY,United States,"[451 Lexington Ave (44th Street), New York, NY...",300304,https://www.seamless.com/menu/grk-fresh-greek-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,130322259
1,5ae8484cd7627e002cc4e055,Greek On 6 Ave,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1593680805,False,NaN,NaN,40.759335,-73.981336,"[{'label': 'display', 'lat': 40.75933511750253...",...,NY,United States,"[New York, NY 10019, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4c0fa77198102d7fa18ae506,Joannne's Amazing Greek Cart,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1593680805,False,NaN,47th St & Park Avenue,40.756834,-73.972084,"[{'label': 'display', 'lat': 40.75683449877979...",...,NY,United States,"[New York, NY, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,GRK Fresh Greek,Greek Restaurant,451 Lexington Ave,44th Street,40.753128,-73.974666,"[{'label': 'display', 'lat': 40.753128, 'lng':...",566,10017,US,New York,NY,United States,"[451 Lexington Ave (44th Street), New York, NY...",5414bab4498e79ad23ecce9b
1,Greek On 6 Ave,Food Truck,NaN,NaN,40.759335,-73.981336,"[{'label': 'display', 'lat': 40.75933511750253...",363,10019,US,New York,NY,United States,"[New York, NY 10019, United States]",5ae8484cd7627e002cc4e055
2,Joannne's Amazing Greek Cart,Food Truck,NaN,47th St & Park Avenue,40.756834,-73.972084,"[{'label': 'display', 'lat': 40.75683449877979...",474,NaN,US,New York,NY,United States,"[New York, NY, United States]",4c0fa77198102d7fa18ae506


In [11]:
dataframe_filtered.name

0                 GRK Fresh Greek
1                  Greek On 6 Ave
2    Joannne's Amazing Greek Cart
Name: name, dtype: object

### Let's visualize the Greek restaurants that are nearby

In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the DZ Bank Building
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Greek restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Get the venue's overall rating

In [19]:
venue_id = '5414bab4498e79ad23ecce9b' # ID of GRK Fresh Greek Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.3


In [16]:
venue_id = '5ae8484cd7627e002cc4e055' # ID of Greek On 6 Ave Greek Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [17]:
venue_id = '4c0fa77198102d7fa18ae506' # ID of Joannne's Amazing Greek Cart Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


### Converting Heffner Agency address to its latitude and longitude coordinates.

In [22]:
address = '40 Wall St, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.707021850000004 -74.00966692266792


### Define a query to search for Greek food that is within 500 metres from the Heffner Agency

In [23]:
search_query = 'Greek'
radius = 500
print(search_query + ' .... OK!')

Greek .... OK!


In [24]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=C0PWVFTOTWSJ0V4CCDVKZOOB2ZNKWS21O2JHHQLFSFRUAXQI&client_secret=LUWI2FWUHN2YRC5HCOYSF24R5JZNTOYTD3XBGSMDD1EC0P5I&ll=40.707021850000004,-74.00966692266792&v=20180604&query=Greek&radius=500&limit=30'

#### Send the GET Request and examine the results

In [ ]:
results = requests.get(url).json()
results

In [26]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,5047c785e4b0bcc0f416cdb3,GRK Fresh Greek - Financial District,"[{'id': '4bf58dd8d48988d10e941735', 'name': 'G...",v-1593681616,False,111 Fulton St,at Dutch St,40.709800,-74.007011,"[{'label': 'display', 'lat': 40.7097995488229,...",...,NY,United States,"[111 Fulton St (at Dutch St), New York, NY 100...",301348,https://www.seamless.com/menu/grk-fresh-greek-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,45901352
1,4e00db2222717718d39ea62f,Ehhnviko Greek Cart,[],v-1593681616,False,Wall Street,William,40.706771,-74.010098,"[{'label': 'display', 'lat': 40.70677067580161...",...,NY,United States,"[Wall Street (William), New York, NY 10005, Un...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5c1be093121384002c345bbf,Greek From Greece (GFG),"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",v-1593681616,False,Broadway,Morris St,40.705935,-74.013091,"[{'label': 'display', 'lat': 40.705935, 'lng':...",...,NY,United States,"[Broadway (Morris St), New York, NY 10004, Uni...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,57ea9dab498e634b49d5663f,Saint Nicholas Greek Orthodox Church and Natio...,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",v-1593681616,False,130 Liberty St,NaN,40.710384,-74.014030,"[{'label': 'display', 'lat': 40.71038436889648...",...,NY,United States,"[130 Liberty St, New York, NY 10006, United St...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5823d69af2c98b32f07ac65a,11 Hanover Greek,"[{'id': '4bf58dd8d48988d10e941735', 'name': 'G...",v-1593681616,False,11 Hanover Sq,NaN,40.705171,-74.009034,"[{'label': 'display', 'lat': 40.70517082398631...",...,NY,United States,"[11 Hanover Sq, New York, NY 10005, United Sta...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,GRK Fresh Greek - Financial District,Greek Restaurant,111 Fulton St,at Dutch St,40.709800,-74.007011,"[{'label': 'display', 'lat': 40.7097995488229,...",381,10038,US,New York,NY,United States,"[111 Fulton St (at Dutch St), New York, NY 100...",5047c785e4b0bcc0f416cdb3
1,Ehhnviko Greek Cart,None,Wall Street,William,40.706771,-74.010098,"[{'label': 'display', 'lat': 40.70677067580161...",45,10005,US,New York,NY,United States,"[Wall Street (William), New York, NY 10005, Un...",4e00db2222717718d39ea62f
2,Greek From Greece (GFG),Bakery,Broadway,Morris St,40.705935,-74.013091,"[{'label': 'display', 'lat': 40.705935, 'lng':...",313,10004,US,New York,NY,United States,"[Broadway (Morris St), New York, NY 10004, Uni...",5c1be093121384002c345bbf
3,Saint Nicholas Greek Orthodox Church and Natio...,Church,130 Liberty St,NaN,40.710384,-74.014030,"[{'label': 'display', 'lat': 40.71038436889648...",525,10006,US,New York,NY,United States,"[130 Liberty St, New York, NY 10006, United St...",57ea9dab498e634b49d5663f
4,11 Hanover Greek,Greek Restaurant,11 Hanover Sq,NaN,40.705171,-74.009034,"[{'label': 'display', 'lat': 40.70517082398631...",212,10005,US,New York,NY,United States,"[11 Hanover Sq, New York, NY 10005, United Sta...",5823d69af2c98b32f07ac65a
5,Absolute Greek Food Truck,Food Truck,NaN,NaN,40.704205,-74.008698,"[{'label': 'display', 'lat': 40.70420476161008...",324,NaN,US,New York,NY,United States,"[New York, NY, United States]",53a458ff498e1d1f0bdd3447


In [28]:
dataframe_filtered.name

0                 GRK Fresh Greek - Financial District
1                                  Ehhnviko Greek Cart
2                              Greek From Greece (GFG)
3    Saint Nicholas Greek Orthodox Church and Natio...
4                                     11 Hanover Greek
5                            Absolute Greek Food Truck
Name: name, dtype: object

### Let's visualize the Greek restaurants that are nearby

In [29]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Heffner Agency
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Greek restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Get the venue's overall rating

In [30]:
venue_id = '5047c785e4b0bcc0f416cdb3' # ID of GRK Fresh Greek Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.8


In [31]:
venue_id = '4e00db2222717718d39ea62f' # ID of Ehhnviko Greek Cart Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [32]:
venue_id = '5c1be093121384002c345bbf' # ID of Greek From Greece Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.6


In [33]:
venue_id = '57ea9dab498e634b49d5663f' # ID of Saint Nicholas Greek Orthodox Church Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [34]:
venue_id = '5823d69af2c98b32f07ac65a' # ID of 11 Hanover Greek Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.6


In [35]:
venue_id = '53a458ff498e1d1f0bdd3447' # ID of Absolute Greek Food Truck Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&clienAbsolute Greek Food Truck
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.
